# Step 2: Generate Single Street Network

Generate one synthetic street network matching a reference city.

**Process:**
1. Load reference data
2. Configure generator
3. Run generation with progress tracking
4. Visualize results and compare to reference
5. Export GeoJSON + metrics

In [ ]:
import sys
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.insert(0, str(Path.cwd()))

from street_network_generator import (
    GeneratorConfig,
    StreetNetworkGenerator,
)
from street_network_generator.metrics import MorphologyMetrics, SpaceSyntaxMetrics
from street_network_generator.visualization import (
    plot_network,
    plot_full_comparison,
    plot_generation_progress,
)
from street_network_generator.validation import NetworkValidator

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

print("✓ Modules loaded")

## Load Reference Data

In [ ]:
# Load saved references from notebook 01
with open('reference_data.pkl', 'rb') as f:
    references = pickle.load(f)

print(f"Loaded {len(references)} reference cities:")
for city in references.keys():
    print(f"  - {city.capitalize()}")

## Configure Generator

Choose target city and generation parameters:

In [ ]:
# CONFIGURATION
TARGET_CITY = 'london'  # Change to: 'london', 'berlin', 'belgrade', 'torino'
SEED = 42
MAX_ITERATIONS = 1500  # Reduce for faster testing

# Load reference
reference = references[TARGET_CITY]

print(f"Target: {TARGET_CITY.capitalize()}")
print(f"Reference nodes: {reference.graph.number_of_nodes()}")
print(f"Reference edges: {reference.graph.number_of_edges()}")
print(f"Node density: {reference.node_density:.1f} nodes/km²")

In [ ]:
# Create configuration
config = GeneratorConfig(
    seed=SEED,
    window_size_m=500,
    max_iterations=MAX_ITERATIONS,
    min_iterations=200,
    syntax_recompute_interval=60,  # Audit every 60 edges
    candidate_per_step=12,
    initial_temp=5.0,
    cooling_rate=0.997,
)

print("\nConfiguration:")
print(f"  Max iterations: {config.max_iterations}")
print(f"  Syntax audit interval: {config.syntax_recompute_interval} edges")
print(f"  Candidates per step: {config.candidate_per_step}")
print(f"  SA temperature: {config.initial_temp} (cooling: {config.cooling_rate})")

## Generate Network

In [ ]:
print("="*70)
print(f" GENERATING NETWORK FOR {TARGET_CITY.upper()}")
print("="*70)

generator = StreetNetworkGenerator(reference, config)
graph, pos, metadata = generator.generate()

print("\n" + "="*70)
print(" GENERATION COMPLETE")
print("="*70)
print(f"Iterations: {metadata['iterations']}")
print(f"Nodes: {graph.number_of_nodes()}")
print(f"Edges: {graph.number_of_edges()}")
print(f"Final score: {metadata['final_score']:.4f}")
print("="*70)

## Visualize Generation Progress

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
plot_generation_progress(metadata['audit_history'], ax=ax)
plt.show()

print(f"\nTotal audits: {len(metadata['audit_history'])}")
print(f"Best score: {min(a['score'] for a in metadata['audit_history']):.4f}")

## Compare: Reference vs Generated

In [ ]:
# Compute metrics for generated network
print("Computing metrics for generated network...")

morph = MorphologyMetrics.compute_all_morphology(
    graph, pos, config.window_size_m
)

syntax = SpaceSyntaxMetrics.compute_all_syntax(graph, radius=3)

print("✓ Metrics computed")

In [ ]:
# Full comparison figure
fig = plot_full_comparison(
    reference, graph, pos, morph, syntax,
    window_size_m=config.window_size_m,
    figsize=(18, 14)
)
plt.show()

## Detailed Metrics Comparison

In [ ]:
import pandas as pd

comparison_data = [
    {
        'Metric': 'Nodes',
        'Reference': reference.graph.number_of_nodes(),
        'Generated': graph.number_of_nodes(),
        'Error': abs(graph.number_of_nodes() - reference.graph.number_of_nodes())
    },
    {
        'Metric': 'Edges',
        'Reference': reference.graph.number_of_edges(),
        'Generated': graph.number_of_edges(),
        'Error': abs(graph.number_of_edges() - reference.graph.number_of_edges())
    },
    {
        'Metric': 'Node Density',
        'Reference': f"{reference.node_density:.2f}",
        'Generated': f"{morph['node_density']:.2f}",
        'Error': f"{abs(morph['node_density'] - reference.node_density):.2f}"
    },
    {
        'Metric': 'Dead-End Ratio',
        'Reference': f"{reference.dead_end_ratio:.3f}",
        'Generated': f"{morph['dead_end_ratio']:.3f}",
        'Error': f"{abs(morph['dead_end_ratio'] - reference.dead_end_ratio):.3f}"
    },
    {
        'Metric': 'Mean Depth',
        'Reference': f"{reference.mean_depth:.3f}",
        'Generated': f"{syntax['mean_depth']:.3f}",
        'Error': f"{abs(syntax['mean_depth'] - reference.mean_depth):.3f}"
    },
    {
        'Metric': 'Intelligibility',
        'Reference': f"{reference.intelligibility:.3f}",
        'Generated': f"{syntax['intelligibility']:.3f}",
        'Error': f"{abs(syntax['intelligibility'] - reference.intelligibility):.3f}"
    },
]

df = pd.DataFrame(comparison_data)
print("\n" + "="*70)
print(f" METRICS COMPARISON: {TARGET_CITY.upper()}")
print("="*70)
print(df.to_string(index=False))
print("="*70)

## Export Results

In [ ]:
# Create output directory
output_dir = Path(f"outputs_generated/{TARGET_CITY}_single")
output_dir.mkdir(parents=True, exist_ok=True)

# Export using validator
validator = NetworkValidator(config)
validator.validate_and_export(
    graph, pos, reference, metadata,
    str(output_dir),
    prefix=f"{TARGET_CITY}_generated"
)

print(f"\n✓ Results exported to: {output_dir}/")

## Save for Batch Comparison

Save this network for comparison with other generated networks:

In [ ]:
# Save network for batch notebook
result = {
    'city': TARGET_CITY,
    'seed': SEED,
    'graph': graph,
    'pos': pos,
    'metadata': metadata,
    'morph': morph,
    'syntax': syntax,
}

with open(f'{TARGET_CITY}_generated_result.pkl', 'wb') as f:
    pickle.dump(result, f)

print(f"✓ Network saved to {TARGET_CITY}_generated_result.pkl")

## Summary

**Next steps:**
1. Try different cities by changing `TARGET_CITY`
2. Adjust parameters (temperature, iterations, etc.)
3. Run notebook 03 to generate 10 networks and compare variability

**Observations:**
- Look at the progress plot: does the score converge?
- Check histogram overlaps: which distributions match best?
- Compare network visualizations: does it "feel" like the reference city?